In [1]:
import header
paths = header.setup_environment()
for k,v in paths.items():
    print(k, '->', v)

import os
import pandas as pd

INPUTDIR = os.path.join(paths['new_data'], '03_merge_inventories')
dfall = pd.read_csv(os.path.join(INPUTDIR, 'all2.csv'))

for subdir in ['Baro', '20hz', '100hz']:
    dfsub = dfall[dfall['subdir']==subdir]
    print(dfsub)

ModuleNotFoundError: No module named 'header'